# Initial exploration with Pivot Table

In [1]:
#Initial notes of potential approaches
### Ticket class affect cabin location? If so might have impact on survival.
### Since ticket class a reflection of socio status, rescue efforts might be skewed.
### Gender already shown with females having disproportionate survival rates, contender for bayesian analysis.
### Children with only nanny affects survival rates?
### Children with siblings affect survival rates?
### Does embarkation port affect allocation of cabin? Might then have an effect on location of cabin and hence survival.
### Age plays a part in survial? Elderly tend to be less rescued / harder to be rescued?
### How does fare relate to ticket class? Better to use fare to reflect socio-economic class?


In [2]:
#Data Exploration with Pivot table
### Fare vs Survival and Pclass vs Survival and Fare vs Pclass 
###### Pclass 1 has a general spread of (5) 25.5875 to 512.3292
###### Pclass 2 has a general spread of 10.05 to 73.5
###### Pclass 3 has a general spread of 4.0125 to 69.55
###### There exist in 3 classes zero fares.
###### Survival in all 3 classes are 136, 87, 119 respectively, 39.77%, 25.44%, 34.8%

#Pclass fares spreads over wide range. May not be adequate to solely use Pclass to decide socio-class. 
#May be good to cluster / use fare ranges / remove outlier fares and use fares to decide Pclass proper.


In [3]:
### Gender vs Age
###### Gender survival rate is 74.2% and 18.89% for female and male respectively

#Good to reduce ages to age ranges to see visualise relationship better.


In [4]:
### Parch and Sibsp
###### Parch = 0 included adults not travelling with children.
###### It is therefore not an indication of children travelling without parents.
###### To identify children without parents, we probably can check parch=0 combined with a age range.
###### They however, may be travelling with a sibling, which can be identified using sibsp, and/ or surname
###### Also, if they are from the same family, likely they will be in the same class and have the same fare.

#Good to try to identify who is in the same group by using surname, embarkation and fare / Ticket and maybe Parch / Sibsp
# This might give a better indication of how having or not having a family affects survival.

In [5]:
### Embarkation
###### Port C survival rate is 55.36% while Q and S is 38.96% and 33.7% respectively.
###### However, we also noted that the intake / sales of classes differs from port to port.
###### Port C sold more class 1 and 2 compared to Port S which sold more class 3.

# Feature Engineering

In [6]:
#Add features to better interpret data and see relationships better
###### Remove fare outliers and cluster fare to identify suitable fare groups,
###### which might indirectly tell / relate socio-economic class and potential cabin location / quality. (window / no window etc)

###### Group age groups into ranges

###### Identify potential family relation, family size, unaccompanied minor / teens
###### Split family name, group ticket / fares, embarkation

###### Impute missing age values and others, considering multiple features, e.g. embarkation, family relation, sibsp, parch etc

###### Check cabin is properly assigned; alpha only or alpha num


In [7]:
import pandas as pd
import numpy as np

In [8]:
titanic_raw_train = pd.read_csv("./titanic_data_raw/train.csv")
titanic_raw_test = pd.read_csv("./titanic_data_raw/test.csv")

In [9]:
titanic_raw_train['label'] = 'train'
titanic_raw_test['label'] = 'test'

In [10]:
titanic_raw_combined = pd.concat([titanic_raw_train, titanic_raw_test])

In [11]:
print("Length of df = ", len(titanic_raw_combined))
titanic_raw_combined.tail()

Length of df =  1309


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,label
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,test
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,test
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,test
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,test
417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,test


### Add family name column

In [12]:
#create family name column
titanic_raw_combined["LastName"] = titanic_raw_combined["Name"].str.split(",", n=1, expand=True)[0]

### Add cabin info columns

In [13]:
# check cabin is properly assigned. Fill NaN cabin to 0 for proper processing.
titanic_raw_combined["Cabin"] = titanic_raw_combined["Cabin"].replace(np.nan,"not_assigned")
titanic_raw_combined["Cabin"] = titanic_raw_combined["Cabin"].astype(str)
titanic_raw_combined["cabin_proper"] = list(map(lambda x: (not x.isalpha()) & (x.strip().isalnum()), titanic_raw_combined['Cabin']))

In [14]:
titanic_raw_combined["multiple_cabin"] = list(map(lambda x: len(x)>1, titanic_raw_combined["Cabin"].str.split(" "))) 

### Impute Embarkation

In [15]:
impute_embarkation_df_full = titanic_raw_combined[['Fare', 'Ticket', 'Embarked', 'Pclass', 'SibSp', 'Parch','label']]

In [16]:
na_mask = impute_embarkation_df_full.isnull()
original = impute_embarkation_df_full
impute_embarkation_df_full = impute_embarkation_df_full[~na_mask]
impute_embarkation_df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Fare      1308 non-null   float64
 1   Ticket    1309 non-null   object 
 2   Embarked  1307 non-null   object 
 3   Pclass    1309 non-null   int64  
 4   SibSp     1309 non-null   int64  
 5   Parch     1309 non-null   int64  
 6   label     1309 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 81.8+ KB


In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

impute_embarkation_df_full['Ticket'] = le.fit_transform(impute_embarkation_df_full['Ticket'])
impute_embarkation_df_full['Embarked'] = le.fit_transform(impute_embarkation_df_full['Embarked'])

In [18]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

imputer_embarked = MissForest()
embarked_fit = imputer_embarked.fit(impute_embarkation_df_full[impute_embarkation_df_full['label']=='train'].drop('label', axis=1))

In [19]:
impute_embarkation_df_full = impute_embarkation_df_full.where(~na_mask, original)
impute_embarkation_df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Fare      1308 non-null   float64
 1   Ticket    1309 non-null   int32  
 2   Embarked  1307 non-null   object 
 3   Pclass    1309 non-null   int64  
 4   SibSp     1309 non-null   int64  
 5   Parch     1309 non-null   int64  
 6   label     1309 non-null   object 
dtypes: float64(1), int32(1), int64(3), object(2)
memory usage: 76.7+ KB


In [20]:
embarked_imputed = imputer_embarked.transform(impute_embarkation_df_full.drop('label', axis=1))

C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has b

Iteration: 0


C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has b

Iteration: 1


C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has b

Iteration: 2


In [21]:
titanic_raw_combined['Embarked'] = le.inverse_transform(embarked_imputed[:, 2].round(0).astype('int64'))
titanic_raw_combined['Fare'] = embarked_imputed[:, 0]

### Add potential family relationship

In [22]:
#is potential family when they have the following in common
### same surname
### same embarkation
### same ticket
### same fare

titanic_raw_combined['family_number'] = titanic_raw_combined.groupby(['LastName', 'Ticket', 'Fare', 'Pclass', 'Embarked']).ngroup()

In [23]:
#count family member. greater than 1 means a family, one means single.
titanic_raw_combined['family_memb_count'] = titanic_raw_combined.groupby(['family_number'])['family_number'].transform('count')

In [24]:
titanic_raw_combined["has_family"] = list(map(lambda x: (x>1), titanic_raw_combined['family_memb_count']))

In [25]:
titanic_raw_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PassengerId        1309 non-null   int64  
 1   Survived           891 non-null    float64
 2   Pclass             1309 non-null   int64  
 3   Name               1309 non-null   object 
 4   Sex                1309 non-null   object 
 5   Age                1046 non-null   float64
 6   SibSp              1309 non-null   int64  
 7   Parch              1309 non-null   int64  
 8   Ticket             1309 non-null   object 
 9   Fare               1309 non-null   float64
 10  Cabin              1309 non-null   object 
 11  Embarked           1309 non-null   object 
 12  label              1309 non-null   object 
 13  LastName           1309 non-null   object 
 14  cabin_proper       1309 non-null   bool   
 15  multiple_cabin     1309 non-null   bool   
 16  family_number      1309 n

### Add potential friendship

In [26]:
#is potential family when they have the following in common
### different surname
### same embarkation
### same ticket
### same fare

titanic_raw_combined['friend_number'] = titanic_raw_combined.groupby(['Ticket', 'Fare', 'Pclass', 'Embarked']).ngroup()
titanic_raw_combined['friends_count'] = titanic_raw_combined.groupby(['friend_number'])['family_number'].transform('count')
titanic_raw_combined['friends_count'] = titanic_raw_combined['friends_count'].replace(np.nan, 0)

In [27]:
has_friend_df = titanic_raw_combined[(titanic_raw_combined['friends_count']>1) & (titanic_raw_combined['has_family']==False)]
has_friend_df['has_friend'] = True 

C:\Users\Personal\AppData\Local\Temp\ipykernel_17384\1677190393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_friend_df['has_friend'] = True


In [28]:
has_friend_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,label,LastName,cabin_proper,multiple_cabin,family_number,family_memb_count,has_family,friend_number,friends_count,has_friend
2,75,1.0,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,...,train,Bing,False,False,88,1,False,105,8,True
3,170,0.0,3,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,...,train,Ling,False,False,558,1,False,105,8,True
4,510,1.0,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,...,train,Lang,False,False,528,1,False,105,8,True
5,644,1.0,3,"Foo, Mr. Choong",male,NaN,0,0,1601,56.4958,...,train,Foo,False,False,310,1,False,105,8,True
8,839,1.0,3,"Chip, Mr. Chang",male,32.0,0,0,1601,56.4958,...,train,Chip,False,False,178,1,False,105,8,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,1264,NaN,1,"Ismay, Mr. Joseph Bruce",male,49.0,0,0,112058,0.0000,...,test,Ismay,False,True,446,1,False,19,2,True
375,1267,NaN,1,"Bowen, Miss. Grace Scott",female,45.0,0,0,PC 17608,262.3750,...,test,Bowen,False,False,108,1,False,826,7,True
390,1282,NaN,1,"Payne, Mr. Vivian Ponsonby",male,23.0,0,0,12749,93.5000,...,test,Payne,True,False,740,1,False,87,4,True
400,1292,NaN,1,"Bonnell, Miss. Caroline",female,30.0,0,0,36928,164.8667,...,test,Bonnell,True,False,99,1,False,627,4,True


In [29]:
titanic_raw_combined = titanic_raw_combined.merge(has_friend_df[['PassengerId','has_friend']], how='left', on='PassengerId')
titanic_raw_combined['has_friend'] = titanic_raw_combined['has_friend'].replace(np.nan, False)

In [30]:
titanic_raw_combined

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,label,LastName,cabin_proper,multiple_cabin,family_number,family_memb_count,has_family,friend_number,friends_count,has_friend
0,546,0.0,1,"Nicholson, Mr. Arthur Ernest",male,64.0,0,0,693,26.0000,...,train,Nicholson,False,False,678,1,False,687,1,False
1,873,0.0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,...,train,Carlsson,False,True,158,1,False,689,1,False
2,75,1.0,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,...,train,Bing,False,False,88,1,False,105,8,True
3,170,0.0,3,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,...,train,Ling,False,False,558,1,False,105,8,True
4,510,1.0,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,...,train,Lang,False,False,528,1,False,105,8,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,...,test,Spector,False,False,884,1,False,713,1,False
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,...,test,Oliva y Ocana,True,False,707,1,False,836,3,True
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,...,test,Saether,False,False,826,1,False,875,1,False
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,...,test,Ware,False,False,966,1,False,580,1,False


In [31]:
len(titanic_raw_combined)

1309

### add no family / friends

In [32]:
titanic_raw_combined = titanic_raw_combined.assign(no_fam_friend = ~titanic_raw_combined.has_family & ~titanic_raw_combined.has_friend)

### drop columns that are not useful

In [33]:
titanic_raw_combined.drop(['family_number', 'friend_number'], axis=1, inplace=True)

### Add age group column

In [34]:
#fill null age considering nultiple factors

In [35]:
#check if miss or master in name
import re
titanic_raw_combined = titanic_raw_combined.assign(has_master_miss = titanic_raw_combined.Name.str.contains("master|miss", flags=re.IGNORECASE))

In [36]:
#create df for imputation
age_impute_df_full = titanic_raw_combined.drop(['PassengerId', 'Name', 'LastName', 'Survived'], axis=1)

In [37]:
age_impute_df_full.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,label,cabin_proper,multiple_cabin,family_memb_count,has_family,friends_count,has_friend,no_fam_friend,has_master_miss
0,1,male,64.0,0,0,693,26.0000,not_assigned,S,train,False,False,1,False,1,False,True,False
1,1,male,33.0,0,0,695,5.0000,B51 B53 B55,S,train,False,True,1,False,1,False,True,False
2,3,male,32.0,0,0,1601,56.4958,not_assigned,S,train,False,False,1,False,8,True,False,False
3,3,male,28.0,0,0,1601,56.4958,not_assigned,S,train,False,False,1,False,8,True,False,False
4,3,male,26.0,0,0,1601,56.4958,not_assigned,S,train,False,False,1,False,8,True,False,False


In [38]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

age_impute_df_full['Sex'] = le.fit_transform(age_impute_df_full['Sex'])
age_impute_df_full['Ticket'] = le.fit_transform(age_impute_df_full['Ticket'])
age_impute_df_full['Cabin'] = le.fit_transform(age_impute_df_full['Cabin'])
age_impute_df_full['Embarked'] = le.fit_transform(age_impute_df_full['Embarked'])

In [39]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

imputer = MissForest()
age_imputed_fit = imputer.fit(age_impute_df_full[age_impute_df_full['label']=='train'].drop('label', axis=1))

In [40]:
age_imputed = imputer.transform(age_impute_df_full.drop('label', axis=1))

C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has b

Iteration: 0
Iteration: 1


C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has b

Iteration: 2
Iteration: 3
Iteration: 4


C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Personal\.conda\envs\tensorflow_gpu\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [41]:
len(age_imputed[:, 2])

1309

In [42]:
titanic_raw_combined['Age'] = age_imputed[:, 2]

In [43]:
#create age range dataframe for mapping
age_map = {"lower_limit":[0.0,3.00001,7.00001,14.00001,18.00001,25.00001,35.00001,45.00001,55.00001,65.00001,75.00001], 
           "upper_limit":[3.0,7.0,14.0,18.0,25.0,35.0,45.0,55.0,65.0,75.0,150.0]}

age_map_df = pd.DataFrame(data=age_map)
age_map_df.reset_index(inplace=True)
age_map_df=age_map_df.rename(columns={'index':'age_grp'}) 
age_map_df['age_grp'] = age_map_df['age_grp'].astype('category')
age_map_df.tail()

,age_grp,lower_limit,upper_limit
6,6,35.00001,45.0
7,7,45.00001,55.0
8,8,55.00001,65.0
9,9,65.00001,75.0
10,10,75.00001,150.0


In [44]:
#join the map to main df and retain only if age conditions match to get age_range group
titanic_raw_combined = titanic_raw_combined.merge(age_map_df, how="cross")

In [45]:
titanic_raw_combined = titanic_raw_combined[(titanic_raw_combined['Age']<=titanic_raw_combined['upper_limit']) & (titanic_raw_combined['Age']>=titanic_raw_combined['lower_limit'])]
titanic_raw_combined.drop(['lower_limit', 'upper_limit'], axis=1, inplace=True)

In [46]:
len(titanic_raw_combined)

1309

### Save as excel for review / analysis using pivot table

In [47]:
titanic_raw_combined.to_csv('./Titanic1/titanic_raw_combined_engineered.csv')

# Model Training

In [48]:
model_data = titanic_raw_combined.drop(['PassengerId', 'Name', 'LastName'], axis=1)

In [49]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

model_data['Sex'] = le.fit_transform(model_data['Sex'])
model_data['Ticket'] = le.fit_transform(model_data['Ticket'])
model_data['Cabin'] = le.fit_transform(model_data['Cabin'])
model_data['Embarked'] = le.fit_transform(model_data['Embarked'])

In [50]:
training_data = model_data[model_data['label']=='train'].drop('label', axis=1)
test_data = model_data[model_data['label']=='test'].drop('label', axis=1)

In [51]:
x_train = training_data.drop('Survived', axis=1)
y_train = training_data['Survived']
x_test = test_data.drop('Survived', axis=1)

In [52]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 8 to 9799
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Pclass             891 non-null    int64   
 1   Sex                891 non-null    int32   
 2   Age                891 non-null    float64 
 3   SibSp              891 non-null    int64   
 4   Parch              891 non-null    int64   
 5   Ticket             891 non-null    int32   
 6   Fare               891 non-null    float64 
 7   Cabin              891 non-null    int32   
 8   Embarked           891 non-null    int32   
 9   cabin_proper       891 non-null    bool    
 10  multiple_cabin     891 non-null    bool    
 11  family_memb_count  891 non-null    int64   
 12  has_family         891 non-null    bool    
 13  friends_count      891 non-null    int64   
 14  has_friend         891 non-null    bool    
 15  no_fam_friend      891 non-null    bool    
 16  has_mas

In [53]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(x_train, y_train);

# Prediction

In [54]:
#test data
survival_pred = rf.predict(x_test)

In [59]:
output_dataset = titanic_raw_combined[titanic_raw_combined['label']=='test'].drop('Survived', axis=1)

In [60]:
x_test['Survived'] = survival_pred

In [61]:
output_dataset = pd.concat([output_dataset, x_test], axis=1)

In [63]:
output_dataset['Survived'] = output_dataset['Survived'].astype('int')

In [64]:
surv_pred_submission = output_dataset[['PassengerId', 'Survived']]
surv_pred_submission.to_csv('./Titanic1/titanic_1_submission.csv', index=False)